# EEG Graph Construction using EEGRAPH

This notebook converts EEG signals into graph representations for graph neural network analysis.

## EEGRAPH Library Features

### Connectivity Measures Implemented
1. **Pearson Correlation**: Linear statistical relationship between channels
2. **Squared Coherence**: Frequency-domain coherence across EEG bands
3. **Cross-Correlation**: Time-domain correlation with lag analysis
4. **Weighted Phase Lag Index (wPLI)**: Phase synchronization robust to volume conduction
5. **Shannon Entropy**: Information-theoretic complexity measure

### Frequency Bands Analyzed
- **Delta** (0.5-4 Hz): Deep sleep, unconscious processes
- **Theta** (4-8 Hz): Drowsiness, meditation, creativity  
- **Alpha** (8-13 Hz): Relaxed awareness, eyes closed
- **Beta** (13-30 Hz): Active thinking, concentration
- **Gamma** (30-100 Hz): Higher cognitive functions

## Processing Pipeline
1. **Data Conversion**: Convert .set files to .edf format for EEGRAPH compatibility
2. **Graph Construction**: Apply multiple connectivity measures
3. **Feature Extraction**: Generate graph-based features for each subject
4. **Multi-Scale Analysis**: Analyze connectivity across different time windows

## Output
- Graph adjacency matrices for each connectivity measure
- Node features representing EEG channel properties
- Multi-band connectivity graphs for comprehensive analysis
- Processed data ready for graph neural network training

In [ ]:
# Install required packages for EEG graph analysis
!pip install EEGRAPH
!pip install EDFlib-Python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 59.6 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of pandas to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import eegraph
import os
from mne.io import read_raw_eeglab

In [ ]:
def create_folder(folder_name):
    """Create a folder if it doesn't exist."""
    os.makedirs(folder_name, exist_ok=True)

def run_analyses(file_path, window):
    """
    Run comprehensive connectivity analysis using EEGRAPH.
    
    Args:
        file_path (str): Path to EEG data file
        window (int): Window size for analysis
    
    Returns:
        tuple: Results from different connectivity measures
    """
    # Initialize EEGRAPH
    G = eegraph.Graph()
    G.load_data(path=file_path)
    
    print(f"Analyzing file: {file_path} with window size: {window}")
    
    # 1. Pearson Correlation
    print("  Computing Pearson correlation...")
    _, output1 = G.modelate(window_size=window, connectivity='pearson_correlation')
    
    # 2. Squared Coherence across frequency bands
    print("  Computing squared coherence...")
    freq_bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    _, output2 = G.modelate(
        window_size=window, 
        connectivity='squared_coherence', 
        bands=freq_bands
    )
    
    # 3. Cross-Correlation
    print("  Computing cross-correlation...")
    _, output4 = G.modelate(window_size=window, connectivity='cross_correlation')
    
    # 4. Weighted Phase Lag Index (wPLI)
    print("  Computing wPLI...")
    _, output6 = G.modelate(
        window_size=window, 
        connectivity='wpli', 
        bands=freq_bands
    )
    
    # 5. Shannon Entropy
    print("  Computing Shannon entropy...")
    _, output8 = G.modelate(window_size=window, connectivity='shannon_entropy')
    
    print("  All connectivity measures completed!")
    
    return output1, output2, output4, output6, output8

In [ ]:
# Convert .set files to .edf format for EEGRAPH compatibility
print("Converting EEG files from .set to .edf format...")

# Setup output directory
output_folder = '/kaggle/working/edf_files'
create_folder(output_folder)

# Convert files for all subjects
conversion_count = 0
for subject_num in range(1, 89):
    # Construct input file path
    file_path = f'/kaggle/input/open-nuro-dataset/dataset/derivatives/sub-{subject_num:03d}/eeg/sub-{subject_num:03d}_task-eyesclosed_eeg.set'
    
    if os.path.isfile(file_path):
        try:
            # Load .set file using MNE
            raw = read_raw_eeglab(file_path, preload=False)
            
            # Define output file path
            output_file = os.path.join(output_folder, f'subject_{subject_num:03d}.edf')
            
            # Export to .edf format
            raw.export(output_file, overwrite=True)
            
            conversion_count += 1
            if conversion_count % 10 == 0:
                print(f"Converted {conversion_count} files...")
                
        except Exception as e:
            print(f"Error converting subject {subject_num:03d}: {str(e)}")
    else:
        print(f"File not found for subject {subject_num:03d}")

print(f"Conversion completed! {conversion_count} files converted to .edf format.")

/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = read_raw_eeglab(file_path)
/tmp/ipykernel_33/395673528.py:22: RuntimeWarning: EDF format requires equal-length data blocks, so 0.2 seconds of zeros were appended to all channels when writing the final block.
  raw.export(output_file,overwrite=True)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = read_raw_eeglab(file_path)


Exported subject 001 to /kaggle/working/edf_files/subject_001.edf


/tmp/ipykernel_33/395673528.py:22: RuntimeWarning: EDF format requires equal-length data blocks, so 0.9 seconds of zeros were appended to all channels when writing the final block.
  raw.export(output_file,overwrite=True)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = read_raw_eeglab(file_path)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_eeglab(file_path)


Exported subject 002 to /kaggle/working/edf_files/subject_002.edf


/tmp/ipykernel_33/395673528.py:22: RuntimeWarning: EDF format requires equal-length data blocks, so 0.9 seconds of zeros were appended to all channels when writing the final block.
  raw.export(output_file,overwrite=True)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = read_raw_eeglab(file_path)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_eeglab(file_path)


...

/tmp/ipykernel_33/395673528.py:22: RuntimeWarning: EDF format requires equal-length data blocks, so 0.2 seconds of zeros were appended to all channels when writing the final block.
  raw.export(output_file,overwrite=True)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = read_raw_eeglab(file_path)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_eeglab(file_path)


Exported subject 086 to /kaggle/working/edf_files/subject_086.edf


/tmp/ipykernel_33/395673528.py:22: RuntimeWarning: EDF format requires equal-length data blocks, so 0.24 seconds of zeros were appended to all channels when writing the final block.
  raw.export(output_file,overwrite=True)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
  raw = read_raw_eeglab(file_path)
/tmp/ipykernel_33/395673528.py:16: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = read_raw_eeglab(file_path)


Exported subject 087 to /kaggle/working/edf_files/subject_087.edf
Exported subject 088 to /kaggle/working/edf_files/subject_088.edf


/tmp/ipykernel_33/395673528.py:22: RuntimeWarning: EDF format requires equal-length data blocks, so 0.3 seconds of zeros were appended to all channels when writing the final block.
  raw.export(output_file,overwrite=True)
